# Tennis Contact Point Analysis - MVP

Upload a tennis rally video to detect contact points, estimate 3D pose, and measure contact position relative to body landmarks.

**Steps:**
1. Run the **Setup** cell to install dependencies
2. Run the **Upload & Process** cell to upload your video and run the pipeline
3. View results and download annotated images + CSV

In [ ]:
#@title 1. Setup - Install dependencies and clone repo
import os, sys, shutil

REPO_URL = "https://github.com/xiaoxiang-ma/tennis_contact_point_spacing.git"
REPO_DIR = "/content/tennis_contact_point_spacing"

# Always re-clone to ensure latest code
if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)
!git clone {REPO_URL} {REPO_DIR}

!pip install -q -r {REPO_DIR}/requirements.txt

# Clear any cached module imports from previous runs
for mod_name in list(sys.modules.keys()):
    if mod_name.startswith(("src.", "utils.")):
        del sys.modules[mod_name]

if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

print("Setup complete!")

In [ ]:
#@title 2. Upload & Process Video
import numpy as np
import pandas as pd
import cv2
from google.colab import files
from tqdm.notebook import tqdm
from IPython.display import display, Image as IPImage
import os

from utils.video_io import load_video
from utils.coordinate_transforms import (
    pelvis_origin_transform, estimate_ground_plane,
    apply_ground_plane, estimate_player_height_scale
)
from src.ball_detection import BallTracker
from src.pose_estimation import PoseEstimator, LANDMARK_MAP
from src.contact_detection import detect_contacts
from src.measurements import compute_measurements
from src.visualization import (
    annotate_contact_frame, save_annotated_frame
)

# --- Upload video ---
print("Upload your tennis video (MP4, MOV, AVI):")
uploaded = files.upload()
video_filename = list(uploaded.keys())[0]
video_path = os.path.join("/content", video_filename)
with open(video_path, "wb") as f:
    f.write(uploaded[video_filename])

# --- Load video ---
print(f"\nLoading video: {video_filename}")
frames, meta = load_video(video_path)
fps = meta["fps"]
print(f"  {len(frames)} frames, {meta['width']}x{meta['height']}, {fps:.1f} fps, {meta['duration_sec']:.1f}s")

# --- Ball detection ---
print("\nDetecting ball...")
tracker = BallTracker()  # HSV fallback (no TrackNet weights)
ball_detections = tracker.detect_all(frames, progress=True)
print(f"  Ball detected in {len(ball_detections)}/{len(frames)} frames")

# --- Pose estimation ---
print("\nEstimating pose...")
pose_estimator = PoseEstimator(static_image_mode=True, model_complexity=2)

wrist_positions = {}  # frame_num -> (x, y) in pixels
pose_cache = {}  # frame_num -> (world_landmarks, raw_result)

# Only process frames near ball detections + some padding
ball_frames = set(d[0] for d in ball_detections)
frames_to_process = set()
for bf in ball_frames:
    for offset in range(-5, 6):
        f = bf + offset
        if 0 <= f < len(frames):
            frames_to_process.add(f)

for i in tqdm(sorted(frames_to_process), desc="Pose estimation"):
    landmarks, result = pose_estimator.process_frame(frames[i])
    if landmarks is not None:
        pose_cache[i] = (landmarks, result)

        # Get pixel-space wrist positions for contact detection
        pixel_lm = pose_estimator.get_pixel_landmarks(result, frames[i].shape)
        if pixel_lm is not None:
            h, w = frames[i].shape[:2]
            rw = pixel_lm.get("right_wrist")
            lw = pixel_lm.get("left_wrist")
            if rw and lw:
                mid_x = w / 2
                if abs(rw[0] - mid_x) > abs(lw[0] - mid_x):
                    wrist_positions[i] = (float(rw[0]), float(rw[1]))
                else:
                    wrist_positions[i] = (float(lw[0]), float(lw[1]))

pose_estimator.close()
print(f"  Pose estimated for {len(pose_cache)} frames")

# --- Contact detection ---
print("\nDetecting contacts...")
contacts = detect_contacts(
    ball_detections, fps,
    wrist_positions=wrist_positions,
    velocity_spike_threshold=1.8,
    wrist_proximity_px=200,
    min_frame_gap=int(fps * 0.5)  # at least 0.5s apart
)
print(f"  Found {len(contacts)} contact(s)")

if not contacts:
    print("\nNo contacts detected. Try adjusting camera angle or lighting.")

# --- Process each contact ---
output_dir = "/content/output"
os.makedirs(output_dir, exist_ok=True)

all_measurements = []

# Create a fresh pose estimator for any missing contact frames
pe2 = PoseEstimator(static_image_mode=True, model_complexity=2)

for contact_frame, confidence in contacts:
    if contact_frame not in pose_cache:
        landmarks, result = pe2.process_frame(frames[contact_frame])
        if landmarks is None:
            print(f"  Frame {contact_frame}: no pose detected, skipping")
            continue
        pose_cache[contact_frame] = (landmarks, result)

    landmarks, raw_result = pose_cache[contact_frame]

    # Determine dominant wrist (the one further extended from pelvis)
    lw = landmarks.get("left_wrist", np.zeros(3))
    rw = landmarks.get("right_wrist", np.zeros(3))
    pelvis = landmarks.get("pelvis", np.zeros(3))
    if np.linalg.norm(rw - pelvis) > np.linalg.norm(lw - pelvis):
        contact_wrist_name = "right_wrist"
        contact_point = rw
    else:
        contact_wrist_name = "left_wrist"
        contact_point = lw

    # Transform to pelvis-centered, ground-adjusted coordinates
    centered = pelvis_origin_transform(landmarks)
    ground_z = estimate_ground_plane(centered)
    adjusted = apply_ground_plane(centered, ground_z)

    # Contact point in same coordinate system
    contact_adjusted = contact_point - pelvis - np.array([0, 0, ground_z])

    # Compute measurements
    meas = compute_measurements(adjusted, contact_adjusted)
    meas["frame_number"] = contact_frame
    meas["timestamp"] = contact_frame / fps
    meas["confidence"] = confidence
    meas["contact_x"] = contact_point[0]
    meas["contact_y"] = contact_point[1]
    meas["contact_z"] = contact_point[2]
    all_measurements.append(meas)

    # Visualize
    pixel_lm = pe2.get_pixel_landmarks(raw_result, frames[contact_frame].shape)
    if pixel_lm is None:
        # Fallback: re-run pose just for pixel landmarks
        _, r2 = pe2.process_frame(frames[contact_frame])
        pixel_lm = pe2.get_pixel_landmarks(r2, frames[contact_frame].shape)

    if pixel_lm:
        annotated = annotate_contact_frame(
            frames[contact_frame], pixel_lm, contact_wrist_name,
            meas, contact_frame, fps
        )
        out_path = os.path.join(output_dir, f"contact_frame_{contact_frame}.png")
        save_annotated_frame(annotated, out_path)
    print(f"  Contact at frame {contact_frame} (t={contact_frame/fps:.2f}s, conf={confidence:.2f})")

pe2.close()

# Save CSV
if all_measurements:
    video_stem = os.path.splitext(video_filename)[0]
    csv_path = os.path.join(output_dir, f"measurements_{video_stem}.csv")
    df = pd.DataFrame(all_measurements)
    first_cols = ["frame_number", "timestamp", "confidence",
                  "contact_x", "contact_y", "contact_z"]
    other_cols = [c for c in df.columns if c not in first_cols]
    df = df[first_cols + sorted(other_cols)]
    df.to_csv(csv_path, index=False)
    print(f"\nMeasurements saved to {csv_path}")

print("\nProcessing complete!")

In [ ]:
#@title 3. Results - View annotated frames & download
import glob
from IPython.display import display, Image as IPImage, HTML
import pandas as pd

output_dir = "/content/output"

# Display annotated images
png_files = sorted(glob.glob(os.path.join(output_dir, "contact_frame_*.png")))
if png_files:
    print(f"Found {len(png_files)} contact frame(s):\n")
    for png_path in png_files:
        print(os.path.basename(png_path))
        display(IPImage(filename=png_path, width=800))
        print()
else:
    print("No annotated frames found.")

# Display measurements table
csv_files = glob.glob(os.path.join(output_dir, "measurements_*.csv"))
if csv_files:
    df = pd.read_csv(csv_files[0])
    print("\nMeasurements:")
    display(df)

# Download links
print("\n--- Download Files ---")
from google.colab import files as colab_files
for f in png_files + csv_files:
    colab_files.download(f)